In [2]:
import numpy as np

In [3]:
from sklearn.decomposition import PCA

In [6]:
from sklearn.linear_model import LogisticRegression

Original Paper 
https://web.stanford.edu/~hastie/Papers/spca_JASA.pdf

Steps in brief:
1. Compute (univariate) standard regression coefficients for each feature.
2. Form a reduced data matrix consisting of only those features whose univariate coefficient exceeds a threshold θ in absolute value (θ is estimated by cross-validation).
3. Compute the first (or first few) principal components of the reduced data matrix.
4. Use these principal component(s) in a regression model to predict the outcome.

In [70]:
class spca_model():
    
    def __init__(model=None, threshold=0, n_components=0):    
        self._model = model
        self._threshold = threshold
        self._n_components = n_components
        
        self._dropouts = None
        self._pca = None
        
    
    def fit_spca_model(self, x, y):
        x_extra_dim = x[:, np.newaxis]  # Change dimension from (x,y) to (x,1,y)
        # x.reshape(x.shape + (1,))
        
        self._dropouts = []
        
        # Fit each feature with the target (predictor) and if the coefficient is less than the threshold, drop it
        for i in range(0, x_extra_dim.shape[2]): # iterate over all the features
            x_one_feature = x_extra_dim[:, :, i]
            self._model.fit(x_one_feature, y)
            
            if (all([abs(self._model.coef_[0]) < self._threshold])): # Return True if True for all values in the list
                self._dropouts.append(i)
                
        if (len(self._dropouts) == x_extra_dim.shape[2]):  # all features have coef less than the threshold
            raise ValueError('Try a smaller threshold')
            
        if (self._n_components > 0):  # calculate the most important n_components
            self._pca = PCA(n_components=self._n_components)
        else: # if n_components is not more than 0. then all components are kept
            self._pca = PCA(n_components=x_extra_dim.shape[2])
        
        x_tranform = self._pca.fit_transform(x_extra_dim[:, 0, :])
        
        self._model = self._model.fit(x_transform, y)
        
        return self

    
    def spca_predict(self, x):
        # delete from x those features which had coef less than the threshold, and were in the dropout set
        trans_x = np.delete(x, self._dropouts, axis=1)  
        
        # the remaining features have been transformed into components using PCA weights 
        trans_x = self._pca.transform(trans_x)
        
        # predict using the spca model
        return self._model.predict(trans_x)
    
    
    def get_num_components(self):
        # return the number of components after the pca fit
        return self._pca.n_components_

    
    def get_components(self):
        # shape : (n_components, n_features)
        # Principal axes in feature space, representing the directions of maximum variance in the data.
        # Should be run only after the PCA has been run on the training dataset
        return self._pca.components_

    
    def get_coefficients(self):
        # return the coefs of the model the data has been fit on
        return self._model.coef_
    
    
    def get_score(self, x, y):
        # return score of the reduced dimensional data on the model
        return self._model.score(x, y)
    


In [ ]:

# Using logistic regression for classification


In [40]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

In [41]:
mnist.data.shape

(70000, 784)

In [44]:
new_data = mnist.data[:, np.newaxis]

In [46]:
new_data.shape

(70000, 1, 784)

In [53]:
new_data.shape[2]

784

In [57]:
new_data[:, 0, :]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [59]:
pca = PCA(100)

In [60]:
pca.fit(mnist.data)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [61]:
pca.n_components

100

In [62]:
pca.n_components_

100

In [66]:
pca.components_[0]

array([ 1.07698810e-17,  2.14050513e-18, -2.43842967e-18, -3.60454767e-19,
       -2.26048704e-19,  2.65594578e-19, -6.53462146e-19, -1.43214610e-19,
       -2.63550259e-20, -2.05856983e-20, -4.51663077e-21,  4.10290663e-20,
       -9.45584905e-07, -3.72373918e-06, -1.83973347e-06, -7.66555611e-08,
        3.26002694e-21, -2.93801376e-22,  4.21504502e-23,  1.72829275e-21,
        5.24714289e-22,  6.18999358e-23, -1.03911382e-21, -2.39744847e-22,
        2.76791920e-22,  6.00879534e-22, -1.08656719e-22,  1.14925958e-22,
        6.67399479e-23, -6.38182197e-23,  3.37696729e-26,  3.46937735e-23,
        2.16824828e-07,  4.29492619e-07,  4.87706215e-06,  1.65229866e-05,
        2.20363253e-05,  3.88808469e-05,  7.40607809e-05,  8.83248604e-05,
        8.08496292e-05,  8.95693486e-05,  1.06176520e-04,  7.93597125e-05,
        3.60812098e-05,  3.08090474e-05,  2.39076583e-05,  3.64534627e-06,
       -2.94191043e-06,  1.88455270e-06,  1.14903054e-06,  4.83408762e-07,
        1.22112301e-24,  

In [68]:

pca.get_covariance()

array([[ 4.28308680e+02, -4.83538412e-28,  6.78896268e-28, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.83538412e-28,  4.28308680e+02, -2.09010917e-28, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.78896268e-28, -2.09010917e-28,  4.28308680e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.28308680e+02,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  4.28308680e+02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  4.28308680e+02]])